# Chapter 19: 特殊方法與運算子重載 - 詳解範例

本檔案包含 4 個深入解析的範例，幫助你理解特殊方法的實際應用。

## 範例 1：自訂列表類別（序列協定完整實作）

**學習目標**：實作完整的序列協定，包括 `__len__`、`__getitem__`、`__setitem__`、`__delitem__`、`__contains__`

In [ ]:
class CustomList:
    """自訂列表類別 - 完整序列協定"""
    
    def __init__(self, initial_data=None):
        """初始化"""
        self.data = list(initial_data) if initial_data else []
    
    def __len__(self):
        """返回長度"""
        return len(self.data)
    
    def __getitem__(self, index):
        """索引存取"""
        return self.data[index]
    
    def __setitem__(self, index, value):
        """索引賦值"""
        self.data[index] = value
    
    def __delitem__(self, index):
        """刪除項目"""
        del self.data[index]
    
    def __contains__(self, item):
        """成員測試"""
        return item in self.data
    
    def append(self, item):
        """新增項目"""
        self.data.append(item)
    
    def __str__(self):
        return f"CustomList({self.data})"
    
    def __repr__(self):
        return f"CustomList({self.data!r})"

# 測試
lst = CustomList([1, 2, 3])
print(f"列表：{lst}")
print(f"長度：{len(lst)}")
print(f"第一個元素：{lst[0]}")

lst[0] = 10
print(f"修改後：{lst}")

print(f"2 in lst: {2 in lst}")

del lst[0]
print(f"刪除後：{lst}")

**關鍵知識點**：
1. 實作 `__len__` 和 `__getitem__` 是最基本的序列協定
2. `__setitem__` 讓物件可以被修改
3. `__contains__` 讓物件支援 `in` 運算子
4. 這些方法組合起來，讓自訂類別像內建列表一樣使用

## 範例 2：複數運算類別（完整運算子重載）

**學習目標**：為複數類別實作所有算術運算子

In [ ]:
class Complex:
    """複數類別 - 完整運算子重載"""
    
    def __init__(self, real, imag):
        self.real = real
        self.imag = imag
    
    def __add__(self, other):
        """加法：(a+bi) + (c+di) = (a+c) + (b+d)i"""
        return Complex(self.real + other.real, self.imag + other.imag)
    
    def __sub__(self, other):
        """減法"""
        return Complex(self.real - other.real, self.imag - other.imag)
    
    def __mul__(self, other):
        """乘法：(a+bi)(c+di) = (ac-bd) + (ad+bc)i"""
        real = self.real * other.real - self.imag * other.imag
        imag = self.real * other.imag + self.imag * other.real
        return Complex(real, imag)
    
    def __eq__(self, other):
        """相等比較"""
        return self.real == other.real and self.imag == other.imag
    
    def __abs__(self):
        """絕對值（模）：|a+bi| = √(a²+b²)"""
        return (self.real**2 + self.imag**2)**0.5
    
    def __str__(self):
        if self.imag >= 0:
            return f"{self.real}+{self.imag}i"
        else:
            return f"{self.real}{self.imag}i"
    
    def __repr__(self):
        return f"Complex({self.real}, {self.imag})"

# 測試
z1 = Complex(3, 4)
z2 = Complex(1, 2)

print(f"z1 = {z1}")
print(f"z2 = {z2}")
print(f"z1 + z2 = {z1 + z2}")
print(f"z1 * z2 = {z1 * z2}")
print(f"|z1| = {abs(z1)}")

**關鍵知識點**：
1. 數學類別非常適合使用運算子重載
2. `__abs__` 讓物件支援 `abs()` 函式
3. 運算子應該返回新物件，不修改原物件
4. `__str__` 可以根據情況調整格式（正負號處理）

## 範例 3：資料庫連線管理器（上下文管理器）

**學習目標**：實作完整的上下文管理器，處理資源管理與例外

In [ ]:
class DatabaseConnection:
    """資料庫連線管理器 - 完整上下文管理器"""
    
    def __init__(self, host, port, database):
        self.host = host
        self.port = port
        self.database = database
        self.connection = None
        self.transaction = None
    
    def __enter__(self):
        """建立連接並開始交易"""
        print(f"連接到 {self.host}:{self.port}/{self.database}")
        # 模擬連接
        self.connection = f"Connection({self.database})"
        print("開始交易...")
        self.transaction = "TRANSACTION"
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        """處理交易提交或回滾，關閉連接"""
        if exc_type is None:
            # 沒有例外：提交交易
            print("提交交易...")
            print("交易成功！")
        else:
            # 有例外：回滾交易
            print(f"發生錯誤：{exc_value}")
            print("回滾交易...")
            print("交易已回滾")
        
        # 無論如何都關閉連接
        print("關閉連接")
        self.connection = None
        self.transaction = None
        
        # 返回 False 讓例外繼續傳播
        return False
    
    def execute(self, query):
        """執行查詢"""
        if not self.connection:
            raise RuntimeError("未連接到資料庫")
        print(f"執行查詢：{query}")
        return f"Result of: {query}"

# 測試 1：正常情況
print("=== 正常交易 ===")
with DatabaseConnection("localhost", 5432, "mydb") as db:
    db.execute("SELECT * FROM users")
    db.execute("UPDATE users SET name='Alice'")

# 測試 2：例外情況
print("\n=== 例外交易 ===")
try:
    with DatabaseConnection("localhost", 5432, "mydb") as db:
        db.execute("SELECT * FROM users")
        raise ValueError("資料驗證失敗！")
        db.execute("這不會執行")
except ValueError as e:
    print(f"外部捕獲例外：{e}")

**關鍵知識點**：
1. `__enter__` 取得資源（連接、開始交易）
2. `__exit__` 釋放資源（提交/回滾、關閉連接）
3. 根據 `exc_type` 判斷是否有例外發生
4. 返回 `False` 讓例外繼續傳播，返回 `True` 會抑制例外
5. 資源清理邏輯應該在 `__exit__` 中，保證一定執行

## 範例 4：計數器類別（綜合應用）

**學習目標**：整合多個特殊方法，建立功能完整的計數器類別

In [ ]:
from functools import total_ordering

@total_ordering
class Counter:
    """計數器類別 - 綜合特殊方法"""
    
    def __init__(self, name="Counter", initial=0):
        self.name = name
        self.value = initial
    
    # 基本表示
    def __str__(self):
        return f"{self.name}: {self.value}"
    
    def __repr__(self):
        return f"Counter('{self.name}', {self.value})"
    
    # 運算子重載
    def __add__(self, other):
        """兩個計數器相加"""
        if isinstance(other, Counter):
            return Counter(f"{self.name}+{other.name}", self.value + other.value)
        else:
            return Counter(self.name, self.value + other)
    
    def __iadd__(self, value):
        """就地加法：counter += 5"""
        self.value += value
        return self
    
    # 比較運算子（使用 @total_ordering）
    def __eq__(self, other):
        if isinstance(other, Counter):
            return self.value == other.value
        return self.value == other
    
    def __lt__(self, other):
        if isinstance(other, Counter):
            return self.value < other.value
        return self.value < other
    
    # 可呼叫
    def __call__(self, increment=1):
        """呼叫時遞增"""
        self.value += increment
        return self.value
    
    # 型態轉換
    def __int__(self):
        return self.value
    
    def __bool__(self):
        return self.value != 0

# 測試
c1 = Counter("點擊次數", 5)
c2 = Counter("瀏覽次數", 10)

print(f"c1 = {c1}")
print(f"c2 = {c2}")

# 運算子
c3 = c1 + c2
print(f"c1 + c2 = {c3}")

c1 += 3
print(f"c1 += 3 → {c1}")

# 比較
print(f"c1 < c2: {c1 < c2}")
print(f"c1 == 8: {c1 == 8}")

# 可呼叫
print(f"c1() = {c1()}")  # 遞增 1
print(f"c1(5) = {c1(5)}")  # 遞增 5

# 型態轉換
print(f"int(c1) = {int(c1)}")
print(f"bool(c1) = {bool(c1)}")

c_zero = Counter("零", 0)
print(f"bool(c_zero) = {bool(c_zero)}")

**關鍵知識點**：
1. `__add__` vs `__iadd__`：前者返回新物件，後者修改並返回 self
2. `@total_ordering` 簡化比較方法的實作
3. `__call__` 讓物件可以像函式一樣呼叫
4. `__int__`、`__bool__` 等型態轉換方法
5. `isinstance()` 用於檢查參數型態，支援多型

## 🎯 總結

通過這 4 個範例，你應該掌握：

1. **序列協定**：`__len__`、`__getitem__` 等讓類別像容器
2. **運算子重載**：為數學物件實作直覺的運算
3. **上下文管理器**：安全管理資源（連接、檔案、交易）
4. **綜合應用**：組合多個特殊方法建立完整功能

**下一步**：完成 `03-practice.ipynb` 的 8 道練習題！